# 13 - Advanced topics - Cement Pavers albedo example

This journal creates a paver underneath the single-axis trackers, and evaluates the improvement for noon, June 17th with and without the pavers for a location in Davis, CA.

![Paver](../images_wiki/AdvancedJournals/Pavers.PNG)

Measurements:
![Paver](../images_wiki/AdvancedJournals/Pavers_Geometry.PNG)

In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'NewMat')

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

try:
    os.stat(testfolder)
except:
    os.mkdir(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\Silvana\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\NewMat


In [2]:
from bifacial_radiance import *   
import numpy as np

In [3]:
timestamp = 4020 # Noon, June 17th. 
simulationname = 'PVEL_Davis'

#Location:
lat = 38.5449 # Davis, CA
lon = -121.7405 # Davis, CA
# MakeModule Parameters
moduletype='60cellmod'
numpanels = 1  # AgriPV site has 3 modules along the y direction (N-S since we are facing it to the south) .
x = 0.95  
y = 1.838
xgap = 0.02# Leaving 2 centimeters between modules on x direction
ygap = 0.0 # 1 - up 
zgap = 0.06 # gap between modules and torquetube.

# Other default values:

# TorqueTube Parameters
axisofrotationTorqueTube=True
torqueTube = False
cellLevelModule = True

numcellsx = 6
numcellsy = 10
xcell = 0.156
ycell = 0.158
xcellgap = 0.015
ycellgap = 0.015

sensorsy = numcellsy   # one sensor per cell

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

# SceneDict Parameters
gcr = 0.33 # m
albedo = 0.2  #'grass'     # ground albedo
hub_height = 1.237 # m  
nMods = 20 # six modules per row.
nRows = 3  # 3 row

azimuth_ang = 90 # Facing east 


demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(lat, lon) # NJ lat/lon 40.0583° N, 74.4057
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,numpanels = numpanels, xgap=xgap, ygap=ygap, cellLevelModuleParams = cellLevelModuleParams)


path = C:\Users\Silvana\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\NewMat
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_CA_Sacramento.Metro.AP.724839_TMY3.epw
 ... OK!
Saving file EPWs\epw_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos

Module Name: 60cellmod
Pre-existing .rad file objects\60cellmod.rad will be overwritten
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.85 %
Module 60cellmod updated in module.json


In [4]:
description = 'Sherman Williams "Chantilly White" acrylic paint'
materialpav = 'sw_chantillywhite'
Rrefl = 0.5
Grefl = 0.5 
Brefl = 0.5
demo.addMaterial(material=materialpav, Rrefl=Rrefl, Grefl=Grefl, Brefl=Brefl, comment=description)

Added material sw_chantillywhite to file ground.rad


In [5]:
demo.gendaylit(timestamp)
tilt = demo.getSingleTimestampTrackerAngle(metdata, timeindex=timestamp, gcr=gcr, 
                                   axis_azimuth=180, axis_tilt=0, 
                                   limit_angle=60, backtrack=True)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'gcr': gcr,'hub_height':hub_height,'azimuth':azimuth_ang, 'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.

01_06_17_13_00
Created PVEL_Davis.oct


### Simulation without Pavers

In [6]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy)
analysis.analysis(octfile, simulationname+"_noPavers", frontscan, backscan)  # compare the back vs front irradiance  
print("Simulation without Pavers Finished")

Linescan in process: PVEL_Davis_noPavers_Front
Linescan in process: PVEL_Davis_noPavers_Back
Saved: results\irr_PVEL_Davis_noPavers.csv
Simulation without Pavers Finished


### Simulation With Pavers

In [7]:
torquetubelength = moduleDict['scenex']*(nMods) 
pitch = demo.moduleDict['sceney']/gcr
startpitch = -pitch * (nRows-1)/2
p_w = 0.947 # m
p_h = 0.092 # m
p_w2 = 0.187 # m
p_h2 = 0.184 # m
offset_w1y = -(p_w/2)+(p_w2/2)
offset_w2y = (p_w/2)-(p_w2/2)

for i in range (0, nRows):    
    name='PAVER'+str(i)
    text='! genbox {} paver{} {} {} {} | xform -t {} {} 0 | xform -t {} 0 0'.format(materialpav, i, 
                                    p_w, torquetubelength, p_h, 
                                    -p_w/2, (-torquetubelength+moduleDict['sceney'])/2.0,
                                    startpitch+pitch*i)
    text += '\r\n! genbox {} paverS1{} {} {} {} | xform -t {} {} 0 | xform -t {} 0 0'.format(materialpav, i,
                                    p_w2, torquetubelength, p_h2, 
                                    -p_w2/2+offset_w1y, (-torquetubelength+moduleDict['sceney'])/2.0,
                                    startpitch+pitch*i)
    text += '\r\n! genbox {} paverS2{} {} {} {} | xform -t {} {} 0 | xform -t {} 0 0'.format(materialpav, i,
                                    p_w2, torquetubelength, p_h2, 
                                    -p_w2/2+offset_w2y, (-torquetubelength+moduleDict['sceney'])/2.0,
                                    startpitch+pitch*i)

    customObject = demo.makeCustomObject(name,text)
    demo.appendtoScene(radfile=scene.radfiles, customObject=customObject, text="!xform -rz 0")


Custom Object Name objects\PAVER0.rad

Custom Object Name objects\PAVER1.rad

Custom Object Name objects\PAVER2.rad


In [8]:
demo.makeOct()

Created PVEL_Davis.oct


'PVEL_Davis.oct'

### rvu -vf views\front.vp -e .01 -pe 0.01 -vp -5 -14 1 -vd 0 0.9946 -0.1040 PVEL_Davis.oct

In [9]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy)
analysis.analysis(octfile, simulationname+"_WITHPavers", frontscan, backscan)  # compare the back vs front irradiance  
print("Simulation WITH Pavers Finished")


Linescan in process: PVEL_Davis_WITHPavers_Front
Linescan in process: PVEL_Davis_WITHPavers_Back
Saved: results\irr_PVEL_Davis_WITHPavers.csv
Simulation WITH Pavers Finished


In [10]:
df_0 = load.read1Result(os.path.join(testfolder, 'results', 'irr_PVEL_Davis_noPavers.csv'))
df_w = load.read1Result(os.path.join(testfolder, 'results', 'irr_PVEL_Davis_WITHPavers.csv'))                        

In [11]:
df_0

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.773069,4.733681e-17,1.331170,1.309268,a9.1.a2.0.0.cellPVmodule.6457,a9.1.a2.0.0.cellPVmodule.2310,973.3497,117.08920,0.120295
1,0.600836,3.679062e-17,1.314889,1.292987,a9.1.a2.1.0.cellPVmodule.6457,a9.1.a2.1.0.cellPVmodule.2310,973.3496,110.12280,0.113138
2,0.428604,2.624444e-17,1.298609,1.276706,a9.1.a2.2.0.cellPVmodule.6457,a9.1.a2.2.0.cellPVmodule.2310,973.3486,103.60970,0.106447
3,0.256372,1.569826e-17,1.282328,1.260426,a9.1.a2.3.0.cellPVmodule.6457,a9.1.a2.3.0.cellPVmodule.2310,973.3477,99.03805,0.101750
4,0.084140,5.152079e-18,1.266047,1.244145,a9.1.a2.4.0.cellPVmodule.6457,a9.1.a2.4.0.cellPVmodule.2310,973.3467,97.12404,0.099783
5,-0.088092,-5.394103e-18,1.249766,1.227864,a9.1.a2.5.0.cellPVmodule.6457,a9.1.a2.5.0.cellPVmodule.2310,973.3458,96.41985,0.099060
6,-0.260325,-1.594028e-17,1.233486,1.211583,a9.1.a2.6.0.cellPVmodule.6457,a9.1.a2.6.0.cellPVmodule.2310,973.3448,102.61520,0.105425
7,-0.432557,-2.648647e-17,1.217205,1.195303,a9.1.a2.7.0.cellPVmodule.6457,a9.1.a2.7.0.cellPVmodule.2310,973.3438,105.10610,0.107984
8,-0.604789,-3.703265e-17,1.200924,1.179022,a9.1.a2.8.0.cellPVmodule.6457,a9.1.a2.8.0.cellPVmodule.2310,973.3428,109.98640,0.112999
9,-0.777021,-4.757883e-17,1.184643,1.162741,a9.1.a2.9.0.cellPVmodule.6457,a9.1.a2.9.0.cellPVmodule.2310,973.3419,119.14170,0.122405


In [12]:
df_w

,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.773069,4.733681e-17,1.331170,1.309268,a9.1.a2.0.0.cellPVmodule.6457,a9.1.a2.0.0.cellPVmodule.2310,973.5498,119.2991,0.122540
1,0.600836,3.679062e-17,1.314889,1.292987,a9.1.a2.1.0.cellPVmodule.6457,a9.1.a2.1.0.cellPVmodule.2310,973.5479,112.9893,0.116059
2,0.428604,2.624444e-17,1.298609,1.276706,a9.1.a2.2.0.cellPVmodule.6457,a9.1.a2.2.0.cellPVmodule.2310,973.5446,110.4395,0.113441
3,0.256372,1.569826e-17,1.282328,1.260426,a9.1.a2.3.0.cellPVmodule.6457,a9.1.a2.3.0.cellPVmodule.2310,973.5413,111.2228,0.114245
4,0.084140,5.152079e-18,1.266047,1.244145,a9.1.a2.4.0.cellPVmodule.6457,a9.1.a2.4.0.cellPVmodule.2310,973.5380,109.4782,0.112454
5,-0.088092,-5.394103e-18,1.249766,1.227864,a9.1.a2.5.0.cellPVmodule.6457,a9.1.a2.5.0.cellPVmodule.2310,973.5347,107.6355,0.110561
6,-0.260325,-1.594028e-17,1.233486,1.211583,a9.1.a2.6.0.cellPVmodule.6457,a9.1.a2.6.0.cellPVmodule.2310,973.5315,112.4386,0.115495
7,-0.432557,-2.648647e-17,1.217205,1.195303,a9.1.a2.7.0.cellPVmodule.6457,a9.1.a2.7.0.cellPVmodule.2310,973.5282,116.0397,0.119195
8,-0.604789,-3.703265e-17,1.200924,1.179022,a9.1.a2.8.0.cellPVmodule.6457,a9.1.a2.8.0.cellPVmodule.2310,973.5249,119.7284,0.122984
9,-0.777021,-4.757883e-17,1.184643,1.162741,a9.1.a2.9.0.cellPVmodule.6457,a9.1.a2.9.0.cellPVmodule.2310,973.5216,128.0146,0.131496


## Improvement in Rear Irradiance

In [18]:
round((df_w['Wm2Back'].mean()-df_0['Wm2Back'].mean())*100/df_0['Wm2Back'].mean(),1)

8.2